In [61]:
!pip install psycopg2

In [62]:
import psycopg2
import pandas as pd

In [63]:
def db_creation():
    #connect to postgres
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=password")
    #create cursor
    cur = conn.cursor()
    #automatic commit after commands
    conn.set_session(autocommit=True)
    
    
    #create carsales database
    cur.execute("DROP DATABASE carsalesdb")
    cur.execute("CREATE DATABASE carsalesdb")
    
    #close connection to postgres default db
    conn.close()

    #connect to carsalesdb
    conn = psycopg2.connect("host=localhost dbname=carsalesdb user=postgres password=password")
    cur = conn.cursor()
    conn.set_session(autocommit=True)

    return cur, conn

In [66]:
cur, conn = db_creation()

In [67]:
#car table creation
car_table = ("""CREATE TABLE IF NOT EXISTS cars(
transaction_id INT PRIMARY KEY,
car_make VARCHAR,
car_model VARCHAR,
car_year INT)""")
cur.execute(car_table)
conn.commit()

In [68]:
#transaction table creation
transaction_table = ("""CREATE TABLE IF NOT EXISTS transactions(
transaction_id INT PRIMARY KEY,
date DATE,
salesperson VARCHAR,
customer VARCHAR)""")
cur.execute(transaction_table)
conn.commit()

In [69]:
#sales table creation
sales_table = ("""CREATE TABLE IF NOT EXISTS sales(
salesperson VARCHAR,
sales_price INT,
commission_rate FLOAT,
commission_earned FLOAT)""")
cur.execute(sales_table)
conn.commit()

In [2]:
#data load
Data = pd.read_csv("car_sales_data.csv")

In [28]:
#cleaning data and creating datasets to put values into tables
Cars_data = Data[['Transaction_id', 'Car_Make', 'Car_Model', 'Car_Year']]
Transactions_data = Data[['Transaction_id', 'Date', 'Salesperson', 'Customer']]
Sales_data = Data[['Salesperson', 'Sales_Price', 'Commission_Rate', 'Commission_Earned']]

In [70]:
#insert data into car table using iteration method
car_table_insert = ("""INSERT INTO cars(
transaction_id,
car_make,
car_model,
car_year)
VALUES (%s, %s, %s, %s)
""")

In [71]:
for i, row in Cars_data.iterrows():
    cur.execute(car_table_insert, list(row))
conn.commit()

In [72]:
#creating csv files from dataframes to load data faster
Transactions_data.to_csv('transactions_data.csv', sep=',', index=False)
Sales_data.to_csv('sales_data.csv', sep=',', index=False)

In [73]:
#loading data to the tables directly from csv file
with open('sales_data.csv', 'r') as f:
    next(f) # Skip the header row.
    cur.copy_from(f, 'sales', sep=',')
conn.commit()

In [74]:
with open('transactions_data.csv', 'r') as f:
    next(f)
    cur.copy_from(f, 'transactions', sep=',')
conn.commit()